<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных соревнования. Знакомиться будем на [данных](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) Scikit-learn по новостям, сначала в режиме бинарной классификации, затем – в многоклассовом режиме. Затем будем классифицировать рецензии к фильмам с сайта IMDB. Наконец, применим Vowpal Wabbit к данным по веб-сессиям. Материала немало, но Vowpal Wabbit того стоит!

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

In [ ]:
def to_int(predicted_labels):
    return [int(i) for i in predicted_labels]

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_notebooks/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [45]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm 
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, SGDClassifier, LogisticRegressionCV
from sklearn.metrics import accuracy_score

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = '../data/capstone_user_identification/kaggle2/'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), index_col='session_id')

In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [6]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [5]:
# ''' ВАШ КОД ЗДЕСЬ '''
y = train_df_400.user_id.values
class_encoder = LabelEncoder()
y_for_vw = class_encoder.fit_transform(y) + 1

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [6]:
sites = ['site' + str(i) for i in range(1, 11)]

In [7]:
def prepare_site_list(matrix):
    return set(el for row in matrix.values for el in row)

In [10]:
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as site_file:
    sites_dist = pickle.load(site_file)

In [11]:
len(sites_dist)

36656

In [12]:
len(prepare_site_list(train_df_400[sites]))

98507

In [13]:
len(prepare_site_list(test_df_400[sites]))

26587

In [8]:
train_test_df = pd.concat([train_df_400, test_df_400])
train_test_df_sites = train_test_df[sites].fillna(0).astype('int')

In [9]:
def get_dense_matrix(matrix):
    site_ids = prepare_site_list(matrix)
    X = matrix.values
    
    i = 0
    data = list()
    col = list()
    rows = list()
    for row in tqdm(X):
        unique, counts = np.unique(row, return_counts=True)
        dic = dict(zip(unique, counts))
        for k in dic:
            if (k == 0):
                continue
            
            data.append(dic[k])
            rows.append(i)
            col.append(k-1)
            
        i += 1
    X_sparse = csr_matrix((data, (rows, col)), shape=(X.shape[0], len(site_ids)))
    return X_sparse

In [10]:
train_test_sparse = get_dense_matrix(train_test_df_sites)

100%|█████████████████████████████████████████████████████████| 229266/229266 [00:11<00:00, 20316.84it/s]


In [13]:
train_test_sparse.shape

(229266, 36657)

In [11]:
# ''' ВАШ КОД ЗДЕСЬ '''
X_train_sparse = train_test_sparse[:len(train_df_400)]
X_test_sparse = train_test_sparse[len(train_df_400):]
# y = ''' ВАШ КОД ЗДЕСЬ '''

In [12]:
print('Total size {}'.format(train_test_sparse.shape))
print('Train size {}'.format(X_train_sparse.shape))
print('Test size {}'.format(X_test_sparse.shape))
print('Y size {}'.format(len(y)))

Total size (229266, 36657)
Train size (182793, 36657)
Test size (46473, 36657)
Y size 182793


### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [13]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [14]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [67]:
def to_vw_format(row, label=None):
    return str(label or '') + ' | ' + ' '.join(str(t) for t in row) + '\n'

In [62]:
to_vw_format(train_df_part.iloc[0].values, y[0])

'653 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713\n'

In [53]:
# ''' ВАШ КОД ЗДЕСЬ '''
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    with open(os.path.join(PATH_TO_DATA, out_file), 'w') as output_file:
        if train:
            for row, label in zip(X, y):
                output_file.write(to_vw_format(row, label))
        else:
            for row in X:
                output_file.write(to_vw_format(row))

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [68]:
%%time
# будет 4 вызова
arrays_to_vw(train_df_part.as_matrix(), y_train_part_for_vw, out_file='train_df_part.vw')

Wall time: 3.83 s


In [69]:
%%time
arrays_to_vw(valid_df.as_matrix(), y_valid_for_vw, out_file='valid_df.vw')

Wall time: 1.84 s


In [78]:
%%time
arrays_to_vw(train_df_400[sites].as_matrix(), y_for_vw, out_file='X_train_sparse.vw')

Wall time: 5.84 s


In [79]:
%%time
arrays_to_vw(test_df_400[sites].as_matrix(), train=False, out_file='X_test_sparse.vw')

Wall time: 1.18 s


In [77]:
print(X_train_sparse.getrow(0).toarray())
len(X_train_sparse.getrow(0).toarray()[0])

[[0 0 0 ..., 0 0 0]]


36657

**Результат должен получиться таким.**

In [7]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [8]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [9]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


In [ ]:
!vw --version

**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [ ]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [22]:
print('train_df_part.vw')
print('valid_df.vw')
print('X_train_sparse.vw')
print('X_test_sparse.vw')

train_df_part.vw
valid_df.vw
X_train_sparse.vw
X_test_sparse.vw


In [ ]:
%%time
!vw --oaa 400 -d train_df_part.vw --passes 3 -c -k -b 26 --random_seed 17 -f vw_model.vw

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [ ]:
%%time
!vw -t -i vw_model.vw -d valid_df.vw -p vw_valid_pred.csv --random_seed 17

**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [78]:
vw_pred = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_valid_pred.csv'))
accuracy_score(y_valid_for_vw, vw_pred)

0.34277325941865128

**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться долго (у меня – 15 минут) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1.**

In [75]:
%%time
logit = LogisticRegression(random_state=17, n_jobs=-1)
logit.fit(X_train_part_sparse, y_train_part)

C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 29min 17s


In [76]:
%%time
sgd_logit = SGDClassifier(random_state=17, n_jobs=-1, loss='log', n_iter=3, tol=None)
sgd_logit.fit(X_train_part_sparse, y_train_part)

C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Wall time: 18.1 s


**Какими получаются доли правильных ответов на отложенной выборке для Vowpal Wabbit, SGD и логистической регрессии? Запишите 3 числа в файл *answer6_1.txt* (в указанном порядке, сначала VW, потом SGD и logit) через пробел, округлив до 3 знаков после запятой.**

In [79]:
vw_valid_acc = accuracy_score(y_valid_for_vw, vw_pred)
sgd_valid_acc = accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse))
logit_valid_acc = accuracy_score(y_valid, logit.predict(X_valid_sparse))

In [80]:
print(vw_valid_acc, sgd_valid_acc, logit_valid_acc)

0.342773259419 0.291075531566 0.362941755717


In [65]:
print(vw_valid_acc, sgd_valid_acc, logit_valid_acc)

0.342773259419 0.291075531566 0.362941755717


In [31]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [61]:
write_answer_to_file("{} {} {}".format(round(vw_valid_acc, 3), round(sgd_valid_acc, 3), round(logit_valid_acc, 3)), 
                     'answer6_1.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [ ]:
%%time
# vw --oaa 400 -d train_df_part.vw --passes 3 -c -k -b 26 --random_seed 17 -f vw_model.vw
!vw --oaa 400 -d X_train_sparse.vw --passes 3 -c -k -b 26 --random_seed 17 -f vw_model_all.vw

**Сделайте прогноз для тестовой выборки.**

In [ ]:
%%time
# vw -t -i vw_model.vw -d valid_df.vw -p vw_valid_pred.csv --random_seed 17
!vw -t -i vw_model_all.vw -d X_test_sparse.vw -p vw_test_pred.csv --random_seed 17

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [33]:
def write_to_submission_file(predicted_labels, out_file, target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels, index = np.arange(1, predicted_labels.shape[0] + 1), columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [57]:
vw_pred = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_test_pred.csv'))
vw_pred = vw_pred - 1
vw_pred = class_encoder.inverse_transform(vw_pred.astype(int))

[ 89  20 310 ...,  42 153  71]


In [58]:
vw_pred

array([224,  48, 795, ..., 107, 387, 179], dtype=int64)

In [59]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [36]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(random_state=17, n_jobs=-1, loss='log', max_iter=3)

In [37]:
%%time
logit.fit(X_train_sparse, y)

C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 48min 4s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [38]:
logit_test_pred = logit.predict(X_test_sparse)

In [39]:
write_to_submission_file(logit_test_pred.astype(int), os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))

In [40]:
%%time
sgd_logit.fit(X_train_sparse, y)

Wall time: 30.2 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [41]:
sgd_logit_test_pred = sgd_logit.predict(X_test_sparse)

In [42]:
write_to_submission_file(sgd_logit_test_pred.astype(int), os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

**Какими получаются доли правильных ответов  Vowpal Wabbit, SGD и логистической регрессии на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4)? Запишите 3 числа в файл *answer6_2.txt* (в указанном порядке, сначала VW, потом SGD и logit) через пробел, округлив до 3 знаков после запятой.**

In [60]:
vw_result = 0.18656
sgd_result = 0.17490
logit_result = 0.19891

In [62]:
write_answer_to_file("{} {} {}".format(round(vw_result, 3), round(sgd_result, 3), round(logit_result, 3)), 'answer6_2.txt')

In [39]:
train_test_df_times = train_test_df[['time%d' % i for i in range(1, 11)]]
train_test_df_times.head(10)

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
1,2014-03-24 15:22:40,2014-03-24 15:22:48,2014-03-24 15:22:48,2014-03-24 15:22:54,2014-03-24 15:22:54,2014-03-24 15:22:55,2014-03-24 15:23:01,2014-03-24 15:23:03,2014-03-24 15:23:04,2014-03-24 15:23:05
2,2014-04-17 14:25:58,2014-04-17 14:25:59,2014-04-17 14:25:59,2014-04-17 14:25:59,2014-04-17 14:25:59,2014-04-17 14:26:01,2014-04-17 14:26:01,2014-04-17 14:26:18,2014-04-17 14:26:47,2014-04-17 14:26:48
3,2014-03-21 10:12:24,2014-03-21 10:12:36,2014-03-21 10:12:54,2014-03-21 10:13:01,2014-03-21 10:13:24,2014-03-21 10:13:36,2014-03-21 10:13:54,2014-03-21 10:14:01,2014-03-21 10:14:06,2014-03-21 10:14:24
4,2013-12-13 09:52:28,2013-12-13 09:54:34,2013-12-13 09:54:34,2013-12-13 09:54:34,2013-12-13 09:54:34,2013-12-13 09:54:34,2013-12-13 09:54:34,2013-12-13 09:54:34,2013-12-13 09:58:19,2013-12-13 09:58:19
5,2013-11-26 12:35:29,2013-11-26 12:35:31,2013-11-26 12:35:31,2013-11-26 12:35:32,2013-11-26 12:35:32,2013-11-26 12:35:32,2013-11-26 12:37:03,2013-11-26 12:37:03,2013-11-26 12:37:03,2013-11-26 12:37:04
6,2014-01-24 14:06:48,2014-01-24 14:06:49,2014-01-24 14:06:49,2014-01-24 14:06:57,2014-01-24 14:06:58,2014-01-24 14:07:14,2014-01-24 14:07:24,2014-01-24 14:07:36,2014-01-24 14:07:36,2014-01-24 14:07:36
7,2013-11-20 14:45:16,2013-11-20 14:45:17,2013-11-20 14:45:18,2013-11-20 14:45:19,2013-11-20 14:45:21,2013-11-20 14:45:22,2013-11-20 14:45:24,2013-11-20 14:45:28,2013-11-20 14:45:28,2013-11-20 14:45:30
8,2013-02-12 08:42:54,2013-02-12 08:42:54,2013-02-12 08:42:55,2013-02-12 08:42:56,NaN,NaN,NaN,NaN,NaN,NaN
9,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17,2013-11-16 13:35:17


In [41]:
%%time
for c in train_test_df_times.columns:
    train_test_df_times[c] = pd.to_datetime(train_test_df_times[c])

C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Wall time: 7.14 s


In [42]:
%%time
start_hour = []
day_of_week = []
month = []

times = ['time%d' % i for i in range(1, 11)]

for _, row in tqdm(train_test_df_times.iterrows()):
    
    sh = np.zeros(24)
    sh[row[times[0]].hour - 1] += 1
    start_hour.append(sh)
    
    dow = np.zeros(7)
    dow[row[times[0]].dayofweek - 1] += 1 
    day_of_week.append(dow)
    
    m = np.zeros(12)
    m[row[times[0]].month - 1] += 1
    month.append(m)

229266it [00:52, 4373.66it/s]


Wall time: 52.4 s


In [82]:
%%time
unique = [[sum(1 for s in np.unique(row.values) if s != 0)] for _, row in train_test_df_sites.iterrows()]
print(len(unique))
print(unique[0])

229266
[2]
Wall time: 39.3 s


In [132]:
%%time
temp_train_test_sparse = hstack((train_test_sparse, csr_matrix(start_hour)))

Wall time: 466 ms


In [133]:
temp_train_test_sparse.shape

(229266, 36681)

In [134]:
X_train_sparse = temp_train_test_sparse.tocsc()[:len(train_df_400)]
X_test_sparse = temp_train_test_sparse.tocsc()[len(train_df_400):]

In [135]:
print('Total size {}'.format(temp_train_test_sparse.shape))
print('Train size {}'.format(X_train_sparse.shape))
print('Test size {}'.format(X_test_sparse.shape))
print('Y size {}'.format(len(y)))

Total size (229266, 36681)
Train size (182793, 36681)
Test size (46473, 36681)
Y size 182793


In [105]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = X_train_sparse[:train_share, :]
valid_df = X_train_sparse[train_share:, :]

In [55]:
%%time
logit = LogisticRegression(C=2.11111111111, random_state=17, n_jobs=-1)
logit.fit(train_df_part, y_train_part)
accuracy_score(y_valid, logit.predict(valid_df))

0.50747656734381263

In [136]:
%%time
logit = LogisticRegression(C=2.11111111111, n_jobs=-1)
logit.fit(X_train_sparse, y)
logit_test_pred = logit.predict(X_test_sparse)
write_to_submission_file(logit_test_pred.astype(int), os.path.join(PATH_TO_DATA, 'logit_400_users_6.csv'))

C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 1h 2min 9s


**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать и описать результаты в проекте. Не перепутайте! 

## Пути улучшения

Что еще можно попробовать в [соревновании](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2):
 - Использовать ранее построенные признаки для улучшения модели
 - Настроить параметры Vowpal Wabbit с hyperopt
 - Попробовать TF-IDF и n-граммы

На следующей, заключительной, неделе мы оформим всю работу над проектом в виде одного файла (`html`, `pdf` или `ipynb`) и будем проверять проекты друг друга.